In [1]:
from datamatch import (
    ThresholdMatcher,
    ColumnsIndex,
    JaroWinklerSimilarity,
    MaxScorer,
    SimSumScorer,
    AlterScorer,
    MultiIndex,
)
from datavalid.spinner import Spinner
import pandas as pd
from lib import clean_column_names
import re

stty: stdin isn't a terminal


In [2]:
def read():
    df = pd.read_csv("../../preprocessing/data/output/output-clean.csv")
    return df

In [3]:
df = read()

full_names = df.first_name.str.cat(df.last_name, sep=" ")

In [4]:
def jarowink_matcher(personnel):
    excel_path = "../data/output/jarowink_matches_test.xlsx"
    per = personnel
    per
    matcher = ThresholdMatcher(
        index=MultiIndex(
            [
                ColumnsIndex(["fc", "lc"]),
            ]
        ),
        scorer=MaxScorer(
            [
                AlterScorer(
                    scorer=SimSumScorer(
                        {
                            "first_name": JaroWinklerSimilarity(),
                            "last_name": JaroWinklerSimilarity(),
                        }
                    ),
                    # but for pairs that have the same name and their name is common
                    values=full_names,
                    # give a penalty of -.2 which is enough to eliminate them
                    alter=lambda score: score - 0.2,
                ),
            ]
        ),
        dfa=per,
    )
    decision = .5
    with Spinner("saving matched clusters to Excel file"):
        matcher.save_clusters_to_excel(excel_path, decision, lower_bound=decision)
    clusters = matcher.get_index_clusters_within_thresholds(decision)
    print("saved %d clusters to %s" % (len(clusters), excel_path))

    return clusters


In [5]:
jarowink_matcher(df)

IndexError: At least one sheet must be visible